In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

!unzip Data_Segformer.zip

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv
from mmseg.core.evaluation import get_palette
import os
import numpy as np
import glob
from PIL import Image
import os.path as osp
import torch.distributed as dist
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (HOOKS, DistSamplerSeedHook, EpochBasedRunner,
                         build_runner, get_dist_info)
import importlib_metadata
from mmcv.utils import build_from_cfg
from mmseg import digit_version
from mmseg.core import DistEvalHook, EvalHook, build_optimizer
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.utils import find_latest_checkpoint, get_root_logger

data_root = r"D:\Compiled"

classes = ('Background', 'Palm_Tree')

palette = [[0, 0, 0], [0, 137, 37]]


config_file = r"C:\Users\0581979618\mmsegmentation\configs\segformer\segformer_mit-b2_512x512_160k_ade20k.py"


In [ ]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
@DATASETS.register_module()

class Palm_trees(CustomDataset):
    """Custom dataset for semantic segmentation.

    An example of file structure is as followed.

    .. code-block:: none

        ├── data
        │   ├── my_dataset
        │   │   ├── img_dir
        │   │   │   ├── train
        │   │   │   │   ├── xxx{img_suffix}
        │   │   │   │   ├── yyy{img_suffix}
        │   │   │   │   ├── zzz{img_suffix}
        │   │   │   ├── val
        │   │   ├── ann_dir
        │   │   │   ├── train
        │   │   │   │   ├── xxx{seg_map_suffix}
        │   │   │   │   ├── yyy{seg_map_suffix}
        │   │   │   │   ├── zzz{seg_map_suffix}
        │   │   │   ├── val

    The img/gt_semantic_seg pair of CustomDataset should be of the same
    except suffix. A valid img/gt_semantic_seg filename pair should be like
    ``xxx{img_suffix}`` and ``xxx{seg_map_suffix}`` (extension is also included
    in the suffix). If split is given, then ``xxx`` is specified in txt file.
    Otherwise, all files in ``img_dir/``and ``ann_dir`` will be loaded.
    Please refer to ``docs/tutorials/new_dataset.md`` for more details.


    Args:
        pipeline (list[dict]): Processing pipeline
        img_dir (str): Path to image directory
        img_suffix (str): Suffix of images. Default: '.png'
        ann_dir (str, optional): Path to annotation directory. Default: None
        seg_map_suffix (str): Suffix of segmentation maps. Default: '.png'
        split (str, optional): Split txt file. If split is specified, only
            file with suffix in the splits will be loaded. Otherwise, all
            images in img_dir/ann_dir will be loaded. Default: None
        data_root (str, optional): Data root for img_dir/ann_dir. Default:
            None.
        test_mode (bool): If test_mode=True, gt wouldn't be loaded.
        ignore_index (int): The label index to be ignored. Default: 255
        reduce_zero_label (bool): Whether to mark label zero as ignored.
            Default: False
    """

    CLASSES = ('BG', 'Tree')

    PALETTE = [[0, 0, 0], [0, 137, 37]]
    

def __init__(self,
             pipeline,
             img_dir,
             img_suffix='.png',
             ann_dir=None,
             seg_map_suffix='.png',
             split=None,
             data_root=None,
             test_mode=False,
             ignore_index=255,
             reduce_zero_label=False):
    self.pipeline = Compose(pipeline)
    self.img_dir = img_dir
    self.img_suffix = img_suffix
    self.ann_dir = ann_dir
    self.seg_map_suffix = seg_map_suffix
    self.split = split
    self.data_root = data_root
    self.test_mode = test_mode
    self.ignore_index = ignore_index
    self.reduce_zero_label = reduce_zero_label

    # join paths if data_root is specified
    # self.img_dir = osp.join(self.data_root, self.img_dir)
    # self.ann_dir = osp.join(self.data_root, self.ann_dir)
    if self.data_root is not None:
        if not osp.isabs(self.img_dir):
            self.img_dir = osp.join(self.data_root, self.img_dir)
            print(self.img_dir)
        if not (self.ann_dir is None or osp.isabs(self.ann_dir)):
            self.ann_dir = osp.join(self.data_root, self.ann_dir)
        if not (self.split is None or osp.isabs(self.split)):
            self.split = osp.join(self.data_root, self.split)
    # print(self.img_dir,'***************************')
    # load annotations
    self.img_infos = self.load_annotations(self.img_dir, self.img_suffix,
                                           self.ann_dir,
                                           self.seg_map_suffix, self.split)

In [ ]:
from mmcv import Config

config_file = r"C:\Users\0581979618\mmsegmentation\configs\segformer\segformer_mit-b2_512x512_160k_ade20k.py"
checkpoint_file = "/home/mbgibril/mmsegmentation/checkpoints/dpt_vit-b16_512x512_160k_ade20k-db31cf52.pth"
cfg = Config.fromfile(config_file)

### Create a config file
In the next step, we need to modify the config for the training. To accelerate the process, we finetune the model from trained weights.

In [ ]:
from mmseg.apis import set_random_seed

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
#cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
#cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
#cfg.model.auxiliary_head.num_classes = 2

# Modify dataset type and path
cfg.dataset_type = 'Palm_trees'
cfg.data_root = data_root

#cfg.optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)

cfg.optimizer = dict(type='Adam', lr=1e-3, weight_decay=0.0001,
                     paramwise_cfg = dict(
                        custom_keys={
                            'head': dict(lr_mult=10.)
                        }
                        ))

cfg.data.samples_per_gpu =2
cfg.data.workers_per_gpu =2
#cfg.img_norm_cfg = dict(
    #mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(512, 512), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'img_dir/train'
cfg.data.train.ann_dir = 'ann_dir/train'
cfg.data.train.pipeline = cfg.train_pipeline

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'img_dir/val'
cfg.data.val.ann_dir = 'ann_dir/val'
cfg.data.val.pipeline = cfg.test_pipeline

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root 
cfg.data.test.img_dir = 'img_dir/test'
cfg.data.test.ann_dir = 'ann_dir/test'
cfg.data.test.pipeline = cfg.test_pipeline


cfg.load_from = checkpoint_file
#cfg.resume_from = ""

# Set up working dir to save files and logs.
cfg.work_dir = "Output"

# runtime settings
cfg.runner = dict(type='IterBasedRunner', max_iters=20000)
#evaluation = dict(interval=2000, metric='mIoU', pre_eval=True)


cfg.runner.max_iters = 100000
cfg.log_config.interval = 10
cfg.evaluation.interval = 2000
cfg.evaluation.metric = 'mIoU'
cfg.checkpoint_config.interval = 2000

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


meta = dict()
meta['config'] = cfg.pretty_text

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

## Build Segmentor and Train

## Visualize the Results

In [ ]:
from mmcv import Config
import random
import time
# Use your trained model

config_file = "segformer_mit-b2_512x512_160k_ade20k.py"
checkpoint_file='iter_46000.pth'


Validation_image_path = 'image'

cfg = Config.fromfile(config_file)

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')



In [ ]:
# get random test image and visualize it with model
import matplotlib.pyplot as plt

images_list = os.listdir(Validation_image_path)
rand_num = random.randint(0, len(images_list))
Path= os.path.join(Validation_image_path, images_list[rand_num])
img = mmcv.imread(Path)

start = time.time()
result = inference_segmentor(model, img)
print('[INFO]timecost: ', time.time() - start)
 

#palette=[[0, 0, 0], [10, 255, 10]] 
palette=[[0, 0, 0], [0, 176, 80]]

plt.figure(figsize=(2, 2))
# visualize the results 
show_result_pyplot(model, img, result, palette,opacity=0.55)
# or save the visualization results to image files
#model.show_result(img, result, out_file='result1111.jpg')

## save Results

In [ ]:
import glob
from tqdm import tqdm
import cv2

config_file = "segformer_mit-b2_512x512_160k_ade20k.py"
checkpoint_file='iter_46000.pth'

output_dir= "Segformer B2"

Images= 'Data'

images = glob.glob(Images+'/*.jpg')

model.cfg = cfg


for image in tqdm(images):
    img = mmcv.imread(image)
    result = inference_segmentor(model, img)
    palette=[[0, 0, 0], [32, 163, 32]]
    # visualize the results 
    out=model.show_result(img, result,palette,opacity=0.55)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(image)),out) #save the result in the same testing folder
    #cv2.imwrite(os.path.join(output_dir, os.path.basename(image),(result[0]*255).astype(np.uint8)))